In [ ]:
!pip install -U langchain==0.0.316

In [ ]:
!pip install -U openai==0.28.1

In [3]:
import langchain
langchain.__version__

'0.0.316'

In [4]:
import openai

openai.__version__

'0.28.1'

In [6]:
import os
os.environ["OPENAI_API_KEY"] = key

from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI()
llm.predict("How are you?")

"I'm an AI language model, so I don't have feelings, but I'm here to help. How can I assist you today?"

In [7]:
from langchain.llms import OpenAI # gpt 3 open ai model

opean_llm = OpenAI(openai_api_key=key)
opean_llm.predict("How are you?")

"\n\nI'm doing well, thank you. How about you?"

In [8]:
from langchain import chat_models # gpt 3.5 chat gpt
chat_model = ChatOpenAI(openai_api_key=key)
chat_model.predict("How are you?")

"As an AI, I don't have feelings, but I'm functioning well. Thank you for asking! How can I assist you today?"

In [9]:
chat_model.predict("What was my previous question?")

"I'm sorry, but as an AI language model, I don't have access to the browsing history or previous questions of users."

## Chains

In [10]:
from langchain.chains import ConversationChain

chain = ConversationChain(llm=chat_model, verbose=True)
chain.run("How are you today?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: How are you today?
AI:

> Finished chain.


"I'm an AI, so I don't have emotions, but I'm functioning perfectly and ready to assist you! How can I help you today?"

In [11]:
chain.run("What was my previous question?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: How are you today?
AI: I'm an AI, so I don't have emotions, but I'm functioning perfectly and ready to assist you! How can I help you today?
Human: What was my previous question?
AI:

> Finished chain.


'Your previous question was "How are you today?"'

here we can see that using `ConversationChain` we can make gpt remember history.

## Prompts

In [12]:
from langchain.prompts import PromptTemplate

template = """
Return all the subcategories of the following category:

Category: {category}

Subcategories:
"""
prompt = PromptTemplate(input_variables=["category"],
                        template=template)
prompt

PromptTemplate(input_variables=['category'], template=' \nReturn all the subcategories of the following category:\n\nCategory: {category}\n\nSubcategories:\n')

In [13]:
from langchain.chains import LLMChain

chain = LLMChain(llm=chat_model,
                 prompt=prompt,
                 verbose=True
                 )
chain.run("Machine Learning")



> Entering new LLMChain chain...
Prompt after formatting:
 
Return all the subcategories of the following category:

Category: Machine Learning

Subcategories:


> Finished chain.


'- Supervised Learning\n- Unsupervised Learning\n- Reinforcement Learning\n- Deep Learning\n- Natural Language Processing\n- Computer Vision\n- Bayesian Methods\n- Decision Trees\n- Support Vector Machines\n- Neural Networks\n- Genetic Algorithms\n- Clustering Algorithms\n- Dimensionality Reduction\n- Ensemble Methods\n- Transfer Learning\n- Time Series Analysis\n- Recommender Systems\n- Anomaly Detection\n- Reinforcement Learning'

Breakdown prompts in to subelements

In [22]:
from langchain.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate
)

system_template = """
You are helpful assistant who generate comma separated list.
A user will only pass a category and you should generate subcategories of that category in a comma separated list.
ONLY return comma separated list and nothing more!
"""

human_template = """
{category}
"""

system_message = SystemMessagePromptTemplate.from_template(system_template)

human_message = HumanMessagePromptTemplate.from_template(human_template)

chat_template = ChatPromptTemplate.from_messages([
    system_message,
    human_message
])

chain = LLMChain(llm=chat_model,
                 prompt=chat_template,
                 verbose=True
                 )
chain.run("Machine Learning")



> Entering new LLMChain chain...
Prompt after formatting:
System: 
You are helpful assistant who generate comma separated list.
A user will only pass a category and you should generate subcategories of that category in a comma separated list.
ONLY return comma separated list and nothing more!

Human: 
Machine Learning


> Finished chain.


'Supervised Learning, Unsupervised Learning, Reinforcement Learning, Deep Learning, Decision Trees, Random Forests, Support Vector Machines, Neural Networks, Clustering, Dimensionality Reduction'

## Output Parser

making output usable into code

In [23]:
from langchain.schema import BaseOutputParser

class CommaSeperatedParser(BaseOutputParser):

  def parse(self, text):
    output = text.strip().split(',')
    output = [o.strip() for o in output]
    return output

chain = LLMChain(llm=chat_model,
                 prompt=chat_template,
                 verbose=True,
                 output_parser=CommaSeperatedParser()
                 )
chain.run("Machine Learning")


> Finished chain.


['Supervised Learning',
 'Unsupervised Learning',
 'Reinforcement Learning',
 'Deep Learning',
 'Natural Language Processing',
 'Computer Vision',
 'Recommendation Systems']

In [24]:
input_list = [
    {'category': 'Food'},
    {'category': 'Country'},
    {'category': 'Colors'}
]  # generating output for multiple inputs

response = chain.apply(input_list)
response



> Entering new LLMChain chain...
Prompt after formatting:
System: 
You are helpful assistant who generate comma separated list.
A user will only pass a category and you should generate subcategories of that category in a comma separated list.
ONLY return comma separated list and nothing more!

Human: 
Food

Prompt after formatting:
System: 
You are helpful assistant who generate comma separated list.
A user will only pass a category and you should generate subcategories of that category in a comma separated list.
ONLY return comma separated list and nothing more!

Human: 
Country

Prompt after formatting:
System: 
You are helpful assistant who generate comma separated list.
A user will only pass a category and you should generate subcategories of that category in a comma separated list.
ONLY return comma separated list and nothing more!

Human: 
Colors


> Finished chain.


[{'text': ['Fruits', 'Vegetables', 'Meat', 'Dairy', 'Grains']},
 {'text': ['Afghanistan',
   'Albania',
   'Algeria',
   'Andorra',
   'Angola',
   'Antigua and Barbuda',
   'Argentina',
   'Armenia',
   'Australia',
   'Austria',
   'Azerbaijan',
   'Bahamas',
   'Bahrain',
   'Bangladesh',
   'Barbados',
   'Belarus',
   'Belgium',
   'Belize',
   'Benin',
   'Bhutan',
   'Bolivia',
   'Bosnia and Herzegovina',
   'Botswana',
   'Brazil',
   'Brunei',
   'Bulgaria',
   'Burkina Faso',
   'Burundi',
   'Cabo Verde',
   'Cambodia',
   'Cameroon',
   'Canada',
   'Central African Republic',
   'Chad',
   'Chile',
   'China',
   'Colombia',
   'Comoros',
   'Congo',
   'Costa Rica',
   'Croatia',
   'Cuba',
   'Cyprus',
   'Czech Republic',
   'Denmark',
   'Djibouti',
   'Dominica',
   'Dominican Republic',
   'East Timor',
   'Ecuador',
   'Egypt',
   'El Salvador',
   'Equatorial Guinea',
   'Eritrea',
   'Estonia',
   'Eswatini',
   'Ethiopia',
   'Fiji',
   'Finland',
   'France',
 

In [25]:
response[2]['text']

['Red',
 'Blue',
 'Green',
 'Yellow',
 'Orange',
 'Purple',
 'Pink',
 'Brown',
 'Black',
 'White']

## Simple Sequences

pipe different chains together